# Pattern 2. Grammar: 4 Battle-tested Examples
## Notebook ideas

1) compare Structured Output and True Grammar Pattern.
2) demonstrate how to use True Grammar Pattern using llama-cpp and outlines.

## Cases + Ideas
* Guaranteed Format and Rule Compliance
* Generate Proper SQL Query
* Fix grammar in your input sentence (generating the correct one on output)
* Understand the underlying rules (math, natural language, query language, etc)

This notebook demonstrates the Grammar Pattern with 4 practical examples and touches the model from different angles:

1. **Insurance Forms** - Complex nested JSON extraction
2. **SQL Query Generation** - Generate valid SQL against real database using Grammar pattern and `outlines`
3. **Pipe-Separated Data** - Extract structured data with strict format
4. **English** - fix the grammar in your sentence

**Key Learning:** Grammar Pattern provides 100% guarantee of valid output format

## Grammar Pattern approaches. Outlines vs Llama-cpp

| Feature | outlines | llama-cpp |
|---------|----------|-----------|
| Model Format | HuggingFace models | GGUF models |
| Precision | FP16/BF16 (full) | Quantized (Q4, Q5, Q8) |
| Model Download | ✅ Auto-downloads | ⚠️ Manual download required |
| Grammar Support | Regex, JSON schema, FSM | ✅ Full BNF grammar |
| Model Size | ~6GB (full precision) | ~2.5GB (Q5 quantization) |
| Token-level Masking | ✅ Yes | ✅ Yes |
| Format Guarantee | ✅ 100% guaranteed | ✅ 100% guaranteed |
| Grammar Violation | ❌ Physically impossible | ❌ Physically impossible |
| Library Ecosystem | transformers + outlines | llama-cpp-python |
| Hardware Optimization | GPU/MPS preferred | CPU optimized |
| Setup Complexity | Medium (pip install) | Medium (download + pip) |
| Best For | Latest models, GPU | Smaller footprint, CPU |
| Memory Usage | Higher (~6-8GB) | Lower (~3-4GB) |
| Integration | Python native | C++ backend |

## Structured output vs Grammar Pattern

|  Feature | Grammar Pattern         | Structured Outputs                |
|----------|-------------------------|-----------------------------------|
|          | (outlines)              | (Azure OpenAI)                    |
| Implementation           | Self-hosted model       | Azure OpenAI API                  |
| Constraint Type          | Token-level grammar     | Schema + parsing                  |
 Safety Guarantee         | ✅ HARD (impossible)     | ⚠️  SOFT (almost impossible, 99%) |
| Grammar Support          | ✅ BNF, regex, FSM       | ❌ Not supported                   |
| Dangerous SQL Blocking   | ✅ Physically blocked    | ⚠️  Relies on prompt              |
| Output Structure         | ⚠️  Text (grammar)      | ✅ Pydantic objects                |
| Setup Complexity         | Medium (install)        | Low (just API)                    |
| Runtime Performance      | Local (fast)            | API call (latency)                |
| Cost                     | Hardware only           | Per-token pricing                 |
| Model Control            | ✅ Full control          | ❌ Server-side only                |


## Installation

In [ ]:
# Install required packages
!pip install openai pydantic python-dotenv transformers torch accelerate pandas outlines

# Optional: For llama-cpp alternative (requires manual model download)
# !pip install llama-cpp-python


## Setup

In [5]:
import os
from openai import AzureOpenAI
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from enum import Enum
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is set
if not os.getenv("AZURE_OPENAI_API_KEY") or not os.getenv("AZURE_OPENAI_ENDPOINT") or not os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"):
    raise ValueError("Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT_NAME environment variable")

# Setup Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# gpt-4o, 4o-mini, 4.1-mini, and others could be used with slightly different results
MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print("✅ Setup complete!")

✅ Setup complete!


---
# Example 1: Insurance Forms - Complex Nested JSON Extraction
## Uses approach named Text Generation Inference (TGI) or Structured Output
### Structured Output: This approach works in OpenAI/Bedrock models using structured output
### Text Generation Inference (TGI): It also works in llama-cpp models through TGI

## Business Problem

1. Different fields, complex nesting
Insurance companies process thousands of claim forms daily. These forms contain:
  - Personal information, field mapping.
  - Multiple incidents (car accidents often have multiple vehicles)
  - Nested damage descriptions
  - Medical records
  - Financial details

2. Make the model less verbal in places where it's not necessary
  - Do not put extra words, just generate requested content.

**Challenge:** Parsing errors cause claim delays. Extra wording

**Solution:** Grammar Pattern with Pydantic Schema guarantees valid structure.

## Example 1. Step 1. Define Complex Insurance Schema

In [ ]:
# Enums for controlled vocabularies
class ClaimType(str, Enum):
    AUTO = "auto"
    HEALTH = "health"
    HOME = "home"
    LIFE = "life"

class IncidentSeverity(str, Enum):
    MINOR = "minor"
    MODERATE = "moderate"
    SEVERE = "severe"
    TOTAL_LOSS = "total_loss"

class InjuryType(str, Enum):
    NONE = "none"
    MINOR = "minor"
    SERIOUS = "serious"
    CRITICAL = "critical"

# Nested structures - No @dataclass needed with BaseModel!
class PersonalInfo(BaseModel):
    full_name: str = Field(description="Full legal name")
    policy_number: str = Field(description="Insurance policy number")
    phone: str = Field(description="Contact phone number")
    email: Optional[str] = Field(default=None, description="Email address")

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class VehicleInfo(BaseModel):
    license_plate: str
    vin: Optional[str] = Field(default=None, description="Vehicle Identification Number")
    make: str = Field(description="Vehicle manufacturer")
    model: str = Field(description="Vehicle model")
    year: int = Field(ge=1900, le=2030, description="Model year")

class DamageItem(BaseModel):
    component: str = Field(description="Damaged component (e.g., 'front bumper', 'windshield')")
    description: str = Field(description="Detailed damage description")
    estimated_cost: float = Field(ge=0, description="Estimated repair cost in USD")

class Injury(BaseModel):
    person_name: str
    injury_type: InjuryType
    description: str
    medical_facility: Optional[str] = Field(default=None, description="Hospital or clinic name")

class Incident(BaseModel):
    incident_location: Address
    severity: IncidentSeverity
    weather_conditions: Optional[str] = Field(default=None)
    description: str = Field(description="Detailed description of what happened")
    police_report_filed: bool = Field(default=False, description="Whether police report was filed")
    police_report_number: Optional[str] = Field(default=None, description="Police report number if filed")
    incident_date: str = Field(description="Date of the incident in the following format (YYYY-MM-DD)")

class AutoIncident(Incident):
    vehicles_involved: List[VehicleInfo] = Field(min_length=1)
    damages: List[DamageItem] = Field(description="List of damages to each vehicle")
    injuries: List[Injury] = Field(default=[], description="List of injuries")
    other_driver_info: Optional[PersonalInfo] = Field(default=None)

# Main claim structure
class InsuranceClaim(BaseModel):
    claim_type: ClaimType
    claimant: PersonalInfo
    incident: AutoIncident
    claim_id: str = Field(description="Unique claim identifier")
    filing_date: str = Field(description="Date claim was filed (YYYY-MM-DD)")
    total_estimated_cost: float = Field(ge=0, description="Total estimated cost in USD")
    priority: Literal["low", "medium", "high", "urgent"] = Field(
        default="medium",
        description="Claim priority level"
    )

print("✅ Complex insurance schema setup is finished!")

## Example 1. Step 2. Sample Insurance Claim Form (Unstructured Text)
The idea is to extract JSON-formatted output which fits our schema to be processed further

In [ ]:
insurance_form_text = """
CLAIM REPORT - AUTO ACCIDENT

Date Filed: January 5, 2026
Claim Reference: CLM-2026-00472

CLAIMANT INFORMATION:
Name: Sarah Johnson
Policy #: POL-847392-AZ
Contact: (555) 123-4567
Email: sarah.johnson@email.com

INCIDENT DETAILS:
Date of Accident: December 28, 2025
Location: 1234 Main Street, Phoenix, Arizona, 85001
Weather: Rainy conditions, reduced visibility
Police Report: Yes, Report #PX-2025-9847

DESCRIPTION:
I was driving northbound on Main Street at approximately 3:30 PM when another vehicle 
ran a red light and struck my vehicle on the passenger side. The impact caused 
significant damage to both vehicles. The other driver admitted fault at the scene.

MY VEHICLE:
2023 Toyota Camry
License Plate: ABC-1234
VIN: 1HGBH41JXMN109186

DAMAGES TO MY VEHICLE:
1. Front passenger door - Major dent and paint damage - Estimated $2,500
2. Rear passenger door - Moderate dent - Estimated $1,800
3. Passenger side mirror - Broken, needs replacement - Estimated $450
4. Front passenger window - Shattered - Estimated $350

OTHER VEHICLE:
2021 Honda Civic
License Plate: XYZ-9876
Driver: Michael Chen
Driver's Policy: POL-293847-CA
Driver's Phone: (555) 987-6543

DAMAGES TO OTHER VEHICLE:
1. Front bumper - Completely destroyed - Estimated $1,200
2. Hood - Crumpled - Estimated $2,000
3. Headlight assembly - Both broken - Estimated $800

INJURIES:
1. Sarah Johnson (me) - Minor whiplash and bruising - Treated at Phoenix General Hospital
2. Passenger Emma Johnson (my daughter, age 8) - Minor cuts from broken glass - 
   Treated at Phoenix General Hospital

SEVERITY ASSESSMENT: Moderate - vehicles drivable but require significant repairs

TOTAL ESTIMATED DAMAGES: $9,100

Priority: HIGH (injuries involved)
"""

print("Sample Insurance Form Loaded")
print("=" * 80)
print(insurance_form_text[:500] + "...")
print("\n📄 This unstructured text needs to be parsed into structured InsuranceClaim object")

## Example 1. Step 3. "Option 1, Pydantic Schema".
### Extract with Pydantic Schema and Grammar Pattern (Guaranteed Valid output)

In [ ]:
def extract_insurance_claim(form_text: str) -> InsuranceClaim:
    """
    Extract insurance claim with Grammar Pattern guarantee.
    """
    system_prompt = """
    You are an expert insurance claim processing system. Extract all relevant information from the claim form.
    
    ## Insurance Form Structure:
    
    The insurance claim form typically contains the following sections:
    
    ### 1. HEADER SECTION
    - Claim ID/Reference number (e.g., "CLM-2026-00472")
    - Filing date (when the claim was submitted)
    
    ### 2. CLAIMANT INFORMATION SECTION
    - Full legal name of the person filing the claim
    - Policy number (format: POL-XXXXXX-XX)
    - Contact phone number (format: (XXX) XXX-XXXX)
    - Email address (optional)
    
    ### 3. INCIDENT DETAILS SECTION
    - Date of accident/incident (YYYY-MM-DD format)
    - Location: full address including street, city, state, and zip code
    - Weather conditions at time of incident (optional)
    - Police report information (whether filed, report number if available)
    - Detailed narrative description of what happened
    
    ### 4. VEHICLES INVOLVED (for auto claims)
    Each vehicle section includes:
    - Year, Make, Model (e.g., "2023 Toyota Camry")
    - License plate number
    - VIN (Vehicle Identification Number) - optional
    - For other vehicles: driver name, driver's policy number, contact info
    
    ### 5. DAMAGES SECTION
    List of damaged components for each vehicle:
    - Component name (e.g., "front bumper", "passenger door", "windshield")
    - Detailed description of the damage
    - Estimated repair cost in USD (numeric value)
    
    ### 6. INJURIES SECTION (if applicable)
    For each injured person:
    - Person's name
    - Type of injury (none, minor, serious, critical)
    - Description of injuries
    - Medical facility where treated (hospital/clinic name) - optional
    
    ### 7. ASSESSMENT SECTION
    - Severity classification (minor, moderate, severe, total_loss)
    - Total estimated damages (sum of all repair costs)
    - Priority level (low, medium, high, urgent) - often based on injuries and severity
    
    ## Extraction Rules:
    
    1. **Be thorough**: Extract ALL vehicles mentioned, even if it's the other driver's vehicle
    2. **Accuracy**: Use exact values from the form - don't approximate or round numbers
    3. **Dates**: Convert all dates to YYYY-MM-DD format
    4. **Costs**: Extract numeric values only, convert to float (remove $, commas)
    5. **Enums**: Map text to correct enum values:
       - Claim type: auto, health, home, life
       - Severity: minor, moderate, severe, total_loss
       - Injury type: none, minor, serious, critical
    6. **Missing data**: Use appropriate defaults:
       - Optional fields can be null
       - Use empty lists [] for injuries if none reported
    7. **Priority**: Infer from context:
       - urgent: life-threatening injuries or total loss
       - high: any injuries or severe damage
       - medium: moderate damage, no injuries
       - low: minor damage only
    
    ## Common Patterns to Watch For:
    
    - "MY VEHICLE" vs "OTHER VEHICLE" sections - both should be included
    - Damage estimates may be listed per item or as subtotals
    - Police report: "Yes" means filed=true, extract the report number
    - Multiple people may be injured in a single incident
    - The claimant's vehicle should be listed first in vehicles_involved
    
    Extract with precision and completeness.
    """
    
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": form_text}
        ],
        response_format=InsuranceClaim,
        temperature=0
    )

    return response.choices[0].message.parsed

print("Extracting insurance claim with enhanced system prompt...")
print("=" * 80)

claim = extract_insurance_claim(insurance_form_text)

print("\n✅ SUCCESSFULLY EXTRACTED!\n")
print(f"Claim ID: {claim.claim_id}")
print(f"Type: {claim.claim_type.value}")
print(f"Priority: {claim.priority}")
print(f"\nClaimant: {claim.claimant.full_name}")
print(f"Policy: {claim.claimant.policy_number}")
print(f"\nIncident Date: {claim.incident.incident_date}")
print(f"Location: {claim.incident.incident_location.street}, {claim.incident.incident_location.city}")
print(f"Severity: {claim.incident.severity.value}")
print(f"Police Report: {'Yes' if claim.incident.police_report_filed else 'No'}")

print(f"\nVehicles Involved: {len(claim.incident.vehicles_involved)}")
for i, vehicle in enumerate(claim.incident.vehicles_involved, 1):
    print(f"  {i}. {vehicle.year} {vehicle.make} {vehicle.model} ({vehicle.license_plate})")

print(f"\nDamages: {len(claim.incident.damages)}")
for i, damage in enumerate(claim.incident.damages, 1):
    print(f"  {i}. {damage.component}: ${damage.estimated_cost:,.2f}")

print(f"\nInjuries: {len(claim.incident.injuries)}")
for i, injury in enumerate(claim.incident.injuries, 1):
    print(f"  {i}. {injury.person_name}: {injury.injury_type.value} - {injury.description[:50]}...")

print(f"\nTotal Estimated Cost: ${claim.total_estimated_cost:,.2f}")

print("\n" + "=" * 80)
print("\n🎯 ENHANCED SYSTEM PROMPT BENEFITS:")
print("  ✅ Explains insurance form structure in detail")
print("  ✅ Provides clear extraction rules")
print("  ✅ Guides on handling edge cases (MY VEHICLE vs OTHER VEHICLE)")
print("  ✅ Specifies date and number formatting")
print("  ✅ Maps common phrases to enum values")
print("  ✅ Improves accuracy with context-specific instructions")

## Example 1. Step 3. "Option 2: Json-Object".
### Compare: JSON Mode With Pydantic Classes + Grammar Pattern
in order to use Json Mode you have to specify response format this way:

`response_format={"type": "json_object"}`

In [ ]:
# Simulate WITHOUT grammar pattern (just JSON mode)
def extract_without_grammar(form_text: str) -> str:
    """
    Extract without schema constraint - just asks for JSON.
    """
    system_prompt = """
    Extract insurance claim information and return it in JSON format.
    Include: claim_id, claimant info, vehicles, damages, injuries, costs.
    Return valid JSON only.
    """
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": form_text}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    
    return response.choices[0].message.content

print("Extracting WITHOUT grammar pattern...")
print("=" * 80)

json_result = extract_without_grammar(insurance_form_text)
print("\nRaw JSON output:")
print(json.dumps(json.loads(json_result), indent=2))

# Try to parse into our schema
print("\n" + "=" * 80)
print("\n⚠️ PROBLEMS WITHOUT GRAMMAR PATTERN:")
try:
    parsed_dict = json.loads(json_result)
    print("✅ Valid JSON")
    
    # But check if it matches our schema
    try:
        claim_obj = InsuranceClaim(**parsed_dict)
        print("✅ Matches InsuranceClaim schema (lucky!)")
    except Exception as e:
        print(f"❌ Does NOT match InsuranceClaim schema!")
        print(f"   Error: {str(e)[:200]}...")
        
except json.JSONDecodeError as e:
    print(f"❌ Invalid JSON: {e}")

## Example 1. Step 4. Export to JSON for downstream systems

In [ ]:
# Export the claim
claim_json = claim.model_dump_json(indent=2)

print("Final Structured Claim (ready for downstream systems):")
print("=" * 80)
print(claim_json)

# Save to file
with open('insurance_claim.json', 'w') as f:
    f.write(claim_json)

print("\n✅ Saved to insurance_claim.json")
print("\n🎯 This JSON is GUARANTEED to be valid and processable!")

---
# Example 2: SQL Query Generation with TRUE BNF Grammar Pattern

## Business Problem

1. Generate Safe queries to fetch natural data using user's instructions.
2. Natural language to SQL system should generate syntactically correct queries:
- No Wrong column names
- Only valid SQL syntax
- Not Malformed WHERE clauses
- No dangerous operations. **DANGEROUS: DELETE, UPDATE, DROP operations from malicious/confused prompts**

**Solution:** Use **TRUE BNF Grammar Pattern** with llama-cpp to guarantee safe, valid SQL.

## What We'll Demonstrate

Unlike Example 1 (Structured Outputs with Azure OpenAI), this example uses:
- **Self-hosted model** (Qwen 2.5-3B in GGUF format)
- **llama-cpp-python** library
- **BNF Grammar** as a hard constraint (using outlines for Grammar)

This is the **REAL Grammar Pattern** from the article you mentioned!

## Step 1: Create Sample Database & Define BNF Grammar

We'll create an in-memory SQLite database and define the BNF grammar for safe SQL.

In [ ]:
import sqlite3
import pandas as pd

# ============================================================================
# Part A: Create Sample Database
# ============================================================================
print("=" * 80)
print("PART A: CREATING SAMPLE DATABASE")
print("=" * 80)

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department TEXT NOT NULL,
    salary REAL NOT NULL,
    hire_date TEXT NOT NULL,
    manager_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    budget REAL NOT NULL,
    location TEXT NOT NULL
)
''')

# Insert sample data
employees_data = [
    (1, 'John Smith', 'Engineering', 95000, '2020-01-15', None),
    (2, 'Sarah Johnson', 'Engineering', 87000, '2021-03-20', 1),
    (3, 'Michael Chen', 'Engineering', 82000, '2022-06-10', 1),
    (4, 'Emily Brown', 'Sales', 78000, '2020-08-01', None),
    (5, 'David Lee', 'Sales', 71000, '2021-11-15', 4),
    (6, 'Maria Garcia', 'Sales', 69000, '2023-02-01', 4),
    (7, 'James Wilson', 'Marketing', 76000, '2021-05-10', None),
    (8, 'Lisa Anderson', 'Marketing', 68000, '2022-09-20', 7),
    (9, 'Robert Taylor', 'HR', 72000, '2020-04-15', None),
    (10, 'Jennifer Martinez', 'HR', 65000, '2023-01-10', 9)
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?)', employees_data)

departments_data = [
    (1, 'Engineering', 500000, 'San Francisco'),
    (2, 'Sales', 300000, 'New York'),
    (3, 'Marketing', 250000, 'Los Angeles'),
    (4, 'HR', 150000, 'Chicago')
]

cursor.executemany('INSERT INTO departments VALUES (?, ?, ?, ?)', departments_data)
conn.commit()

print("✅ Sample database created!")
print("\nTables:")
print("  • employees (id, name, department, salary, hire_date, manager_id)")
print("  • departments (id, name, budget, location)")

print("\nSample data:")
print(pd.read_sql_query("SELECT * FROM employees LIMIT 3", conn))
print("\n", pd.read_sql_query("SELECT * FROM departments", conn))

# ============================================================================
# Part B: Define BNF Grammar for Safe SQL
# ============================================================================
print("\n" + "=" * 80)
print("PART B: DEFINING BNF GRAMMAR FOR SAFE SQL")
print("=" * 80)

# BNF Grammar for Safe SQL (SELECT only)
SQL_BNF_GRAMMAR = """
root ::= ws select_statement ws

select_statement ::= "SELECT" ws select_list ws from_clause (ws where_clause)? (ws group_by_clause)? (ws order_by_clause)? (ws limit_clause)? ";"?

select_list ::= "*" | column_list
column_list ::= column_expr (ws "," ws column_expr)*
column_expr ::= (identifier ".")? identifier (ws "AS" ws identifier)?
            | aggregate_func "(" ws (column_expr | "*") ws ")" (ws "AS" ws identifier)?

aggregate_func ::= "COUNT" | "SUM" | "AVG" | "MIN" | "MAX"

from_clause ::= "FROM" ws table_ref (ws join_clause)*
table_ref ::= identifier (ws "AS"? ws identifier)?

join_clause ::= join_type ws "JOIN" ws table_ref ws "ON" ws condition
join_type ::= "INNER" | "LEFT" | "RIGHT" | "FULL"

where_clause ::= "WHERE" ws condition

condition ::= simple_condition (ws logic_op ws simple_condition)*
simple_condition ::= column_expr ws comparison_op ws value
                  | column_expr ws "BETWEEN" ws value ws "AND" ws value
                  | column_expr ws "IN" ws "(" ws value_list ws ")"
                  | "(" ws condition ws ")"

comparison_op ::= "=" | "!=" | "<" | ">" | "<=" | ">="
logic_op ::= "AND" | "OR"

group_by_clause ::= "GROUP BY" ws column_list (ws "HAVING" ws condition)?

order_by_clause ::= "ORDER BY" ws order_expr (ws "," ws order_expr)*
order_expr ::= column_expr (ws ("ASC" | "DESC"))?

limit_clause ::= "LIMIT" ws number (ws "OFFSET" ws number)?

value ::= number | string_literal | identifier
value_list ::= value (ws "," ws value)*
string_literal ::= "'" [^']* "'"

identifier ::= [a-zA-Z_][a-zA-Z0-9_]*
number ::= [0-9]+ ("." [0-9]+)?

ws ::= [ \t\n]*
"""

print("\n📝 BNF Grammar defined!")
print("\n🔒 What this grammar enforces:")
print("   ✅ root ::= ws select_statement ws")
print("      → ONLY SELECT statements allowed")
print("      → DELETE, UPDATE, INSERT, DROP are NOT in the grammar")
print("      → Model CANNOT generate these tokens (logit = -inf)")
print("\n   ✅ Supports:")
print("      • WHERE clauses with conditions")
print("      • JOINs (INNER, LEFT, RIGHT, FULL)")
print("      • GROUP BY with HAVING")
print("      • ORDER BY with ASC/DESC")
print("      • LIMIT and OFFSET")
print("      • Aggregate functions (COUNT, SUM, AVG, MIN, MAX)")
print("\n   ✅ Blocks:")
print("      • DELETE (not in grammar)")
print("      • UPDATE (not in grammar)")
print("      • INSERT (not in grammar)")
print("      • DROP, TRUNCATE, ALTER, CREATE (not in grammar)")

print("\n💡 Key Insight:")
print("   Since 'DELETE', 'UPDATE', etc. don't appear anywhere in the BNF,")
print("   llama-cpp will set their token probabilities to -inf (impossible).")
print("   This is TRUE token-level enforcement!")

print("\n" + "=" * 80)
print("Database and Grammar are ready!")
print("=" * 80)

## Step 3: Load Self-Hosted Model which supports grammar

Now we'll load the model and use TRUE Grammar Pattern with BNF constraints.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from outlines import from_transformers
import torch

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

print("=" * 80)
print("LOADING MODEL WITH OUTLINES + HUGGINGFACE")
print("=" * 80)

print("\n📚 Using 'outlines' library for TRUE Grammar Pattern")
print("   • Supports BNF grammar, regex, and JSON schema constraints")
print("   • Token-level logits masking based on grammar rules")
print("   • Same approach used by HuggingFace TGI")
print("   • Works directly with HuggingFace transformers")

# Get HuggingFace token from environment (optional for non-gated models)
hf_token = os.getenv("HF_TOKEN")

print(f"\nLoading model: {MODEL_NAME}...")
print("This may take a few minutes on first run (downloads model)...\n")

# Step 1: Load HuggingFace tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=hf_token)

hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for efficiency
    device_map="auto",  # Automatically use GPU/MPS if available
    token=hf_token
)

print("✅ HuggingFace model loaded")

# Step 2: Wrap with outlines for grammar support
print("\n⏳ Wrapping model with outlines...")
model = from_transformers(hf_model, tokenizer)

print("\n✅ Model loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Device: {hf_model.device}")
print(f"   dtype: {hf_model.dtype}")

print("\n" + "=" * 80)
print("Now we can use TRUE Grammar constraints!")
print("=" * 80)


/Users/alekseikolesnikov/miniconda3/envs/deeplearning-ai/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


LOADING MODEL WITH OUTLINES + HUGGINGFACE

📚 Using 'outlines' library for TRUE Grammar Pattern
   • Supports BNF grammar, regex, and JSON schema constraints
   • Token-level logits masking based on grammar rules
   • Same approach used by HuggingFace TGI
   • Works directly with HuggingFace transformers

Loading model: Qwen/Qwen2.5-3B-Instruct...
This may take a few minutes on first run (downloads model)...



`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ HuggingFace model loaded

⏳ Wrapping model with outlines...

✅ Model loaded successfully!
   Model: Qwen/Qwen2.5-3B-Instruct
   Device: mps:0
   dtype: torch.bfloat16

Now we can use TRUE Grammar constraints!


## Step 2. Test 1. Generate SQL with TRUE BNF Grammar Constraint
Now we'll use the BNF grammar to generate SQL queries with HARD constraints.

In [ ]:
from outlines import Generator, regex

print("=" * 80)
print("CREATING SQL GRAMMAR WITH OUTLINES (REGEX)")
print("=" * 80)

print("\n🔧 Grammar Type: Regex (Regular Expression)")
print("   • Constrains output to match specific patterns")
print("   • Token-level logits masking (tokens that don't match = -inf)")
print("   • TRUE Grammar Pattern (not just prompt engineering)")

# Define SQL regex pattern that only allows SELECT statements
sql_regex = r"SELECT[\s\S]+FROM[\s\S]+"

print("\n📝 SQL Regex Pattern:")
print(f"   {sql_regex}")

print("\n🔒 This regex enforces:")
print("   ✅ Must start with SELECT")
print("   ✅ Must have FROM clause")
print("   ❌ IMPOSSIBLE: DELETE, UPDATE, INSERT, DROP, ALTER")
print("      (These keywords are not in the regex, so tokens are blocked)")

# Create generator with regex grammar
print("\n⏳ Compiling regex grammar...")
sql_generator = Generator(model, output_type=regex(sql_regex))
print("✅ Grammar compiled!")

print("\n💡 How it works:")
print("   1. outlines compiles regex to finite state machine (FSM)")
print("   2. At each token generation:")
print("      → FSM determines which tokens are valid")
print("      → Invalid tokens get logit = -inf (impossible to generate)")
print("      → Model MUST choose valid token")
print("   3. Output GUARANTEED to match regex")

print("\n" + "=" * 80)
print("TESTING: Generate SQL Queries with Grammar Constraint")
print("=" * 80)

# Test questions - SAFE requests
test_questions = [
    "Show me all employees in Engineering department",
    "What is the average salary by department?",
    "List employees who earn more than $75,000"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {question}")
    print("-" * 80)
    
    # Create prompt
    prompt = f"""Generate a SQL SELECT query.

Database Schema:
- employees: id, name, department, salary, hire_date, manager_id
- departments: id, name, budget, location

Question: {question}

SQL Query:\n"""
    
    print("\n⏳ Generating with regex grammar constraint...")
    
    try:
        # Generate with grammar constraint
        sql_query = sql_generator(prompt)
        
        print(f"\n✅ Generated SQL:")
        print(f"   {sql_query}")
        
        # Try to execute it
        try:
            # Clean the query
            clean_sql = sql_query.split(';')[0].strip()
            if not clean_sql.upper().startswith('SELECT'):
                clean_sql = 'SELECT' + clean_sql
                
            df = pd.read_sql_query(clean_sql, conn)
            print(f"\n📊 Execution: SUCCESS")
            print(f"   Rows returned: {len(df)}")
            if len(df) > 0:
                print(f"\n   Results preview:")
                print(df.head().to_string(index=False))
        except Exception as e:
            print(f"\n⚠️ Execution error: {str(e)[:150]}")
            print("   Note: Grammar ensures SELECT-only, but not perfect SQL syntax")
    except Exception as e:
        print(f"\n❌ Generation failed: {str(e)[:200]}")

print("\n" + "=" * 80)


## Step 3. Test 2. SQL Query generation Test. BNF Grammar Block Dangerous Operations?

Let's try to make the model generate dangerous SQL operations.

In [ ]:
print("=" * 80)
print("TESTING GRAMMAR SAFETY: Attempting Dangerous Operations")
print("=" * 80)

# Dangerous requests that should be blocked
dangerous_questions = [
    "Delete all employees from the Engineering department",
    "Update all salaries to $100,000",
    "Drop the employees table"
]

for i, question in enumerate(dangerous_questions, 1):
    print(f"\n{'='*80}")
    print(f"⚠️ Dangerous Request {i}: {question}")
    print("-" * 80)
    
    prompt = f"""Generate SQL for this request.

Database Schema:
- employees: id, name, department, salary, hire_date, manager_id
- departments: id, name, budget, location

Request: {question}

SQL Query:\n"""
    
    print("\n⏳ Attempting to generate with grammar constraint...")
    
    try:
        # Generate with grammar constraint
        # The regex ONLY allows SELECT, so dangerous operations are impossible
        generated_sql = sql_generator(prompt)
        
        print(f"\n📝 What the model generated:")
        print(f"   {generated_sql}")
        
        # Check if it's a SELECT
        if generated_sql.strip().upper().startswith("SELECT"):
            print(f"\n✅ SAFE: Grammar forced a SELECT query instead!")
            print(f"   The model CANNOT generate DELETE/UPDATE/DROP with this regex.")
            print(f"   Even when explicitly asked, grammar physically blocks it.")
        else:
            print(f"\n⚠️ Unexpected: Not a SELECT query")
            print(f"   This shouldn't happen with the regex constraint.")
    
    except Exception as e:
        print(f"\n❌ Generation failed: {str(e)[:150]}")
        print("   This might happen if the model can't satisfy the grammar")

print("\n" + "=" * 80)
print("🛡️ GRAMMAR PATTERN SAFETY DEMONSTRATED:")
print("=" * 80)
print("\n✅ Dangerous SQL operations were BLOCKED by grammar constraint")
print("   • DELETE, UPDATE, INSERT, DROP are NOT in the regex pattern")
print("   • Tokens for these keywords get logit = -inf (impossible)")
print("   • Model physically CANNOT generate them")
print("   • This is TRUE Grammar Pattern - hard constraint at token level")

print("\n🎯 Key Difference from Prompt Engineering:")
print("   ❌ Prompt: 'Only generate SELECT' → Model might ignore")
print("   ✅ Grammar: regex only allows SELECT → Physically impossible to violate")


---
## Alternative: Using llama-cpp-python with BNF Grammar

The `llama-cpp-python` library provides an alternative way to use TRUE BNF Grammar Pattern.

**Key Differences:**
- `outlines`: Works with HuggingFace models (FP16/BF16), auto-downloads
- `llama-cpp`: Works with GGUF models (quantized), requires manual download

**Both provide TRUE Grammar Pattern** with token-level logits masking.

### When to use llama-cpp:
- Need smaller model size (quantized GGUF)
- Want faster inference on CPU
- Prefer BNF grammar over regex

### Setup:
```bash
# Install llama-cpp-python
pip install llama-cpp-python

# Download GGUF model (example)
# Visit: https://huggingface.co/Qwen/Qwen2.5-3B-Instruct-GGUF
# Download: qwen2.5-3b-instruct-q5_k_m.gguf (~2.5GB)
```

In [ ]:
!pip install llama-cpp-python

In [ ]:
# OPTIONAL: llama-cpp-python implementation
# Uncomment to use this alternative approach

from llama_cpp import Llama, LlamaGrammar
from huggingface_hub import hf_hub_download
import os

print("=" * 80)
print("LOADING MODEL WITH LLAMA-CPP + BNF GRAMMAR")
print("=" * 80)

# Create models directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Model details
REPO_ID = "Qwen/Qwen2.5-3B-Instruct-GGUF"
FILENAME = "qwen2.5-3b-instruct-q5_k_m.gguf"
MODEL_PATH = f"models/{FILENAME}"

# Download if not exists
if not os.path.exists(MODEL_PATH):
    print(f"\n📥 Downloading {FILENAME} (~2.5GB)...")
    print("   This may take 5-10 minutes depending on your connection...")
    MODEL_PATH = hf_hub_download(
        repo_id=REPO_ID,
        filename=FILENAME,
        local_dir="models",
        local_dir_use_symlinks=False
    )
    print("✅ Download complete!")
else:
    print(f"\n✅ Model already exists at {MODEL_PATH}")

print(f"\n📦 Loading model from: {MODEL_PATH}")
print("   This may take 30-60 seconds...\n")

# Load model with llama-cpp
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,           # Context window
    n_threads=8,          # Use multiple threads
    n_gpu_layers=0,       # 0 = CPU only, increase for Metal/MPS
    verbose=False
)
    
print("✅ Model loaded successfully!")
    
# Create BNF grammar
print("\n⏳ Compiling BNF grammar...")
sql_grammar = LlamaGrammar.from_string(SQL_BNF_GRAMMAR)
print("✅ Grammar compiled!")
    
# Test generation
print("\n" + "=" * 80)
print("TESTING: Generate SQL with llama-cpp BNF Grammar")
print("=" * 80)
    
test_question = "What is the average salary by department?"
    
prompt = f"""Generate a SQL SELECT query.
Database Schema:
- employees: id, name, department, salary, hire_date, manager_id
- departments: id, name, budget, location

Question: {test_question}

SQL Query:\n"""
    
print(f"\nQuestion: {test_question}")
print("\n⏳ Generating with BNF grammar constraint...")
    
# Generate with grammar constraint
response = llm(
    prompt,
    max_tokens=150,
    temperature=0.3,
   grammar=sql_grammar,  # <-- BNF Grammar constraint!
    stop=[";", "\n\n"]
)
    
sql_query = response['choices'][0]['text'].strip()
    
print(f"\n✅ Generated SQL:")
print(f"   {sql_query}")
    
# Try to execute
try:
    df = pd.read_sql_query(sql_query, conn)
    print(f"\n📊 Execution: SUCCESS")
    print(f"   Rows returned: {len(df)}")
    if len(df) > 0:
        print(f"\n   Results:")
        print(df.to_string(index=False))
except Exception as e:
    print(f"\n⚠️ Execution error: {str(e)[:150]}")

print("\n" + "=" * 80)
print("✅ Both outlines and llama-cpp provide TRUE Grammar Pattern!")
print("   See comparison table below for detailed differences.")
print("=" * 80)

## Implementation Comparison: outlines vs llama-cpp

Both libraries provide TRUE Grammar Pattern with token-level logits masking.

| Feature | outlines | llama-cpp |
|---------|----------|-----------|
| Model Format | HuggingFace models | GGUF models |
| Precision | FP16/BF16 (full) | Quantized (Q4, Q5, Q8) |
| Model Download | ✅ Auto-downloads | ⚠️ Manual download required |
| Grammar Support | Regex, JSON schema, FSM | ✅ Full BNF grammar |
| Model Size | ~6GB (full precision) | ~2.5GB (Q5 quantization) |
| Token-level Masking | ✅ Yes | ✅ Yes |
| Format Guarantee | ✅ 100% guaranteed | ✅ 100% guaranteed |
| Grammar Violation | ❌ Physically impossible | ❌ Physically impossible |
| Library Ecosystem | transformers + outlines | llama-cpp-python |
| Hardware Optimization | GPU/MPS preferred | CPU optimized |
| Setup Complexity | Medium (pip install) | Medium (download + pip) |
| Best For | Latest models, GPU | Smaller footprint, CPU |
| Memory Usage | Higher (~6-8GB) | Lower (~3-4GB) |
| Integration | Python native | C++ backend |

**Bottom Line:** Both provide TRUE Grammar Pattern. Choose based on your infrastructure:
- **outlines**: When you want latest HuggingFace models with auto-download
- **llama-cpp**: When you need smaller model size and CPU efficiency

---
## Example 3. Pipe-separated extraction

Extract Named Entities from the text in the following format:
`SKU | Product Name | Price | Category`
---

In [1]:
# Sample product descriptions
product_descriptions = [
    """
    Apple iPhone 17 Pro - Latest flagship smartphone with Pro chip,
    48MP camera system, and titanium design. SKU: IPHONE-17-PRO-256.
    Price: $999. Category: Electronics/Smartphones
    """,
    """
    Samsung 65" QLED 4K Smart TV - Quantum HDR, Object Tracking Sound+, 
    and Gaming Hub. Model: QN65Q80C. Retail price: $1,299.99
    Category: Electronics/TVs
    """,
    """
    Nike Air Max 270 - Men's running shoes with visible Max Air cushioning.
    Style code: DM9652-001. Price $160. Category: Footwear/Athletic
    """,
    """
    Organic Green Tea - Premium loose leaf tea from Japan. 
    No SKU assigned yet. Price: $24.99 per package. Category: Groceries/Beverages
    """
]

print("Sample Product Descriptions:")
print("=" * 80)
for i, desc in enumerate(product_descriptions, 1):
    print(f"\n{i}. {desc.strip()[:100]}...")

print("\n" + "=" * 80)
print("\n🎯 Goal: Extract as pipe-separated format:")
print("   SKU | Product Name | Price | Category")
print("\nConstraints:")
print("  • Exactly 3 pipes (4 fields)")
print("  • Use NULL for missing SKU")
print("  • Only alphanumeric and basic punctuation")
print("  • Price must be numeric")

Sample Product Descriptions:

1. Apple iPhone 17 Pro - Latest flagship smartphone with Pro chip,
    48MP camera system, and titanium...

2. Samsung 65" QLED 4K Smart TV - Quantum HDR, Object Tracking Sound+, 
    and Gaming Hub. Model: QN65...

3. Nike Air Max 270 - Men's running shoes with visible Max Air cushioning.
    Style code: DM9652-001. ...

4. Organic Green Tea - Premium loose leaf tea from Japan. 
    No SKU assigned yet. Price: $24.99 per p...


🎯 Goal: Extract as pipe-separated format:
   SKU | Product Name | Price | Category

Constraints:
  • Exactly 3 pipes (4 fields)
  • Use NULL for missing SKU
  • Only alphanumeric and basic punctuation
  • Price must be numeric


## Define Pipe-Separated Schema

For pipe-separated format, we'll use Pydantic schema approach (simpler than BNF for this case).

In [3]:
from pydantic import BaseModel, Field

class ProductRecord(BaseModel):
    """Single product in structured format (we'll convert to pipe-separated)"""
    sku: str = Field(description="Product SKU, use 'NULL' if not available")
    product_name: str = Field(description="Product name, alphanumeric only")
    price: float = Field(ge=0, description="Product price in USD")
    category: str = Field(description="Product category")
    
    def to_pipe_format(self) -> str:
        """Convert to pipe-separated format"""
        return f"{self.sku}|{self.product_name}|{self.price:.2f}|{self.category}"

def extract_product_record(description: str) -> ProductRecord:
    """
    Extract product information with Grammar Pattern.
    """
    system_prompt = """
    Extract product information from the description.
    Use 'NULL' for SKU if not provided.
    Clean product name to alphanumeric characters only.
    """
    
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": description}
        ],
        response_format=ProductRecord,
        temperature=0
    )
    
    return response.choices[0].message.parsed

print("✅ Pipe-separated extraction ready!")

✅ Pipe-separated extraction ready!


## Extract Products in Pipe-Separated Format

In [6]:
print("Extracting Products in Pipe-Separated Format")
print("=" * 80)

pipe_records = []

for i, description in enumerate(product_descriptions, 1):
    print(f"\n{'='*80}")
    print(f"\n📦 Product {i}:")
    print("-" * 80)
    print(description.strip()[:150] + "...")
    
    # Extract with Grammar Pattern
    product = extract_product_record(description)
    
    # Convert to pipe format
    pipe_format = product.to_pipe_format()
    pipe_records.append(pipe_format)
    
    print(f"\n✅ Extracted (structured):")
    print(f"   SKU: {product.sku}")
    print(f"   Name: {product.product_name}")
    print(f"   Price: ${product.price:.2f}")
    print(f"   Category: {product.category}")
    
    print(f"\n📄 Pipe-separated format:")
    print(f"   {pipe_format}")

print("\n" + "=" * 80)
print("\n📋 FINAL OUTPUT (ready for legacy system import):")
print("=" * 80)
for record in pipe_records:
    print(record)

print("\n🎯 GUARANTEES:")
print("  ✅ Exactly 3 pipes in each line")
print("  ✅ Price always numeric (float)")
print("  ✅ NULL used for missing SKU")
print("  ✅ No parsing errors")
print("  ✅ Ready for direct import to legacy system")

Extracting Products in Pipe-Separated Format


📦 Product 1:
--------------------------------------------------------------------------------
Apple iPhone 17 Pro - Latest flagship smartphone with Pro chip,
    48MP camera system, and titanium design. SKU: IPHONE-17-PRO-256.
    Price: $999. ...

✅ Extracted (structured):
   SKU: IPHONE-17-PRO-256
   Name: Apple iPhone 17 Pro
   Price: $999.00
   Category: Electronics/Smartphones

📄 Pipe-separated format:
   IPHONE-17-PRO-256|Apple iPhone 17 Pro|999.00|Electronics/Smartphones


📦 Product 2:
--------------------------------------------------------------------------------
Samsung 65" QLED 4K Smart TV - Quantum HDR, Object Tracking Sound+, 
    and Gaming Hub. Model: QN65Q80C. Retail price: $1,299.99
    Category: Electr...

✅ Extracted (structured):
   SKU: QN65Q80C
   Name: Samsung65QLED4KSmartTV
   Price: $1299.99
   Category: Electronics/TVs

📄 Pipe-separated format:
   QN65Q80C|Samsung65QLED4KSmartTV|1299.99|Electronics/TVs


📦 Product

## Save to File

In [ ]:
# Save to PSV (Pipe-Separated Values) file
with open('products.psv', 'w') as f:
    f.write("SKU|Product Name|Price|Category\n")  # Header
    for record in pipe_records:
        f.write(record + "\n")

print("✅ Saved to products.psv")
print("\nFile contents:")
with open('products.psv', 'r') as f:
    print(f.read())

print("\n🎯 This file is GUARANTEED to be:")
print("  ✅ Properly formatted")
print("  ✅ Importable by legacy systems")
print("  ✅ No manual validation needed")

---
# Example 4: English Grammar Correction

## Business Problem

Grammar correction tools need to:
- Fix grammatical errors in user input
- Preserve the original meaning
- Output ONLY the corrected sentence (not explanations)
- Handle various types of errors (subject-verb agreement, tense, articles, etc.)

**Challenge:** LLMs naturally want to explain their corrections or add commentary.

**Solution:** Grammar Pattern constrains output to be valid English sentence only.

## What We'll Demonstrate

We'll use **regex grammar** with `outlines` to ensure:
1. Output is a single sentence
2. Starts with capital letter
3. Ends with proper punctuation
4. Contains only valid characters (no explanations, no bullet points)


## Define English Sentence Grammar

We'll use regex to constrain the output to valid English sentences:

```regex
^[A-Z][A-Za-z0-9\s,.'"\-!?]+[.!?]$
```

**What this regex enforces:**
- `^[A-Z]` - Must start with capital letter
- `[A-Za-z0-9\s,.'"\-!?]+` - Contains letters, numbers, spaces, and basic punctuation
- `[.!?]$` - Must end with sentence-ending punctuation

**What it prevents:**
- Multi-sentence responses
- Explanations (e.g., "The error was...")
- Bullet points or formatting
- Missing capitalization or punctuation


In [9]:
from outlines import Generator, regex

print("=" * 80)
print("CREATING ENGLISH SENTENCE GRAMMAR")
print("=" * 80)

# Regex for valid English sentence
# - Starts with capital letter
# - Contains alphanumeric, spaces, and basic punctuation
# - Ends with . ! or ?
english_sentence_regex = r"[A-Z][A-Za-z0-9\s,.'\"\-!?]*[.!?]"

print("\n📝 English Sentence Regex:")
print(f"   {english_sentence_regex}")

print("\n🔒 This regex enforces:")
print("   ✅ Starts with capital letter")
print("   ✅ Valid characters only (letters, numbers, basic punctuation)")
print("   ✅ Ends with proper punctuation (. ! ?)")
print("   ❌ BLOCKS: Multi-sentence responses, explanations, formatting")

print("\n⏳ Creating grammar-constrained generator...")
grammar_corrector = Generator(model, output_type=regex(english_sentence_regex))
print("✅ Grammar generator created!")

print("\n" + "=" * 80)


CREATING ENGLISH SENTENCE GRAMMAR

📝 English Sentence Regex:
   [A-Z][A-Za-z0-9\s,.'\"\-!?]*[.!?]

🔒 This regex enforces:
   ✅ Starts with capital letter
   ✅ Valid characters only (letters, numbers, basic punctuation)
   ✅ Ends with proper punctuation (. ! ?)
   ❌ BLOCKS: Multi-sentence responses, explanations, formatting

⏳ Creating grammar-constrained generator...
✅ Grammar generator created!



## Test: Grammar Correction with Constrained Output

In [10]:
# Sample sentences with grammatical errors
test_sentences = [
    "She don't like pizza.",  # Subject-verb agreement
    "He go to school yesterday.",  # Tense error
    "I have a apple and orange.",  # Article error
    "They was happy about the news.",  # Subject-verb agreement
    "Me and him went to store.",  # Pronoun case error
    "The cat it is sleeping on the chair.",  # Redundant pronoun
    "She have three brother and two sister.",  # Multiple errors
]

In [11]:
print("=" * 80)
print("CORRECTING GRAMMAR WITH CONSTRAINED OUTPUT")
print("=" * 80)

for i, incorrect_sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*80}")
    print(f"Example {i}:")
    print("-" * 80)
    print(f"❌ Original: {incorrect_sentence}")
    
    # Create prompt
    prompt = f"""Fix the grammar in this sentence. Output ONLY the corrected sentence, nothing else.

Incorrect: {incorrect_sentence}

Corrected: """
    
    print("\n⏳ Generating with grammar constraint...")
    
    try:
        # Generate with grammar constraint
        corrected = grammar_corrector(prompt)
        
        print(f"✅ Corrected: {corrected}")
        
        # Verify it matches our regex
        import re
        if re.match(english_sentence_regex, corrected):
            print("   ✅ Matches grammar constraints")
        else:
            print("   ⚠️  Does not match regex (shouldn't happen!)")
            
    except Exception as e:
        print(f"❌ Generation failed: {str(e)[:150]}")

print("\n" + "=" * 80)
print("🎯 GRAMMAR PATTERN BENEFITS:")
print("=" * 80)
print("\n✅ Output is ALWAYS a valid sentence:")
print("   • Starts with capital letter")
print("   • Ends with proper punctuation")
print("   • No extra explanations or commentary")
print("   • Single sentence only (no paragraphs)")
print("\n✅ Physical constraint at token level:")
print("   • Model CANNOT generate invalid characters")
print("   • Model CANNOT skip capitalization or punctuation")
print("   • 100% guarantee of format compliance")


CORRECTING GRAMMAR WITH CONSTRAINED OUTPUT

Example 1:
--------------------------------------------------------------------------------
❌ Original: She don't like pizza.

⏳ Generating with grammar constraint...
✅ Corrected: She doesn't like pizza. She doesn't like pizza. Great! Here's the corrected sentence.
   ✅ Matches grammar constraints

Example 2:
--------------------------------------------------------------------------------
❌ Original: He go to school yesterday.

⏳ Generating with grammar constraint...
✅ Corrected: He went to school yesterday. To correct the sentence, "He go to school yesterday," we need
   ✅ Matches grammar constraints

Example 3:
--------------------------------------------------------------------------------
❌ Original: I have a apple and orange.

⏳ Generating with grammar constraint...
✅ Corrected: Correction needed. The corrected sentence is missing. Could you please provide the correct version? I have an
   ✅ Matches grammar constraints

Example 4:
------

In [ ]:
test_sentence = "She don't like pizza."

print("=" * 80)
print("COMPARISON: Grammar Pattern vs Regular Prompting")
print("=" * 80)
print(f"\nTest sentence: {test_sentence}")

# Without Grammar Pattern (using API)
print("\n" + "=" * 80)
print("❌ WITHOUT GRAMMAR PATTERN (regular prompting):")
print("-" * 80)

response_no_grammar = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a grammar checker. Fix errors in user sentences."},
        {"role": "user", "content": f"Fix the grammar: {test_sentence}"}
    ],
    temperature=0
)

print(response_no_grammar.choices[0].message.content)

print("\n⚠️ Problems:")
print("   • May include explanation ('The error is...')")
print("   • May format with quotes or markdown")
print("   • May include multiple variations")
print("   • Inconsistent output format")
print("   • Hard to parse programmatically")

# With Grammar Pattern
print("\n" + "=" * 80)
print("✅ WITH GRAMMAR PATTERN (regex constraint):")
print("-" * 80)

prompt = f"""Fix the grammar in this sentence. Output ONLY the corrected sentence.

Incorrect: {test_sentence}

Corrected: """

corrected = grammar_corrector(prompt)
print(corrected)

print("\n✅ Benefits:")
print("   • Always just the corrected sentence")
print("   • No explanations or formatting")
print("   • Consistent output structure")
print("   • Easy to parse and use")
print("   • Physical guarantee (not just prompt)")

print("\n" + "=" * 80)
print("💡 KEY INSIGHT:")
print("=" * 80)
print("\nRegular prompting (soft constraint):")
print("   ⚠️  'Please only output the sentence' → Model might ignore")
print("   ⚠️  Relies on model's instruction following")
print("   ⚠️  Can be bypassed or misunderstood")
print("\nGrammar Pattern (hard constraint):")
print("   ✅ Regex physically blocks invalid tokens")
print("   ✅ Model CANNOT generate explanations")
print("   ✅ 100% guaranteed format compliance")
print("\n🎯 Use Grammar Pattern when output format MUST be exact!")
